In [1]:
!pip install transformers
!pip install --upgrade tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
from transformers import InputExample, InputFeatures
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

In [32]:
import tensorflow as tf
import pandas as pd
df = pd.read_json('review_1000.json', lines=True)

#get only the text
#sentences = df.text.values

#assigns positive of negative (words) to sentiment
df["Sentiment"] = df.stars.apply(lambda score: "positive" if score >= 3 else "negative")
# change positive / negative to 1/ 0
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})
df['ratings'] = df.stars
#just show the review and the new 1/0 
df = df[["text", "Sentiment", "ratings"]]

print(df)

                                                  text  Sentiment  ratings
0    If you decide to eat here, just be aware it is...          1        3
1    I've taken a lot of spin classes over the year...          1        5
2    Family diner. Had the buffet. Eclectic assortm...          1        3
3    Wow!  Yummy, different,  delicious.   Our favo...          1        5
4    Cute interior and owner (?) gave us tour of up...          1        4
..                                                 ...        ...      ...
995  I Recently vacationed in St. Petersburg with m...          1        5
996  Very cool bar attached to the Alexander Hotel....          1        4
997  This hotel is a gem in the heart of Philadelph...          1        5
998  Was taken here today by a Colombian friend who...          1        5
999  My friends and I all got one of each that were...          1        4

[1000 rows x 3 columns]


In [33]:
reviews = df.text.values.tolist()
labels = df.Sentiment.tolist()
stars = df.ratings.tolist()



In [7]:
print(reviews[:2])
print(labels[:2])

["If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", "I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycle. From the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.\n\nFor anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gyms make you do).\n\nThere is no way I can write this review without giving Russell, th

In [8]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [9]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


In [10]:
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=128)

{'input_ids': [[101, 1045, 2293, 2023, 2173, 1012, 1012, 1012, 2644, 7188, 1045, 2064, 1025, 2593, 2012, 2023, 3295, 2030, 1996, 4407, 2028, 1012, 1045, 2071, 4521, 1996, 13988, 2378, 2806, 12559, 2296, 2154, 1997, 1996, 2733, 1012, 2021, 9826, 1010, 2035, 1997, 2037, 2833, 2003, 2204, 1012, 1045, 1005, 1049, 5791, 1037, 5470, 1997, 2037, 10442, 10439, 20624, 6290, 1012, 2111, 2064, 7475, 2055, 2129, 14469, 3973, 20980, 2009, 2003, 1012, 1012, 1012, 3649, 1012, 1996, 2069, 2173, 1045, 1005, 2310, 2941, 2018, 20980, 2833, 5510, 3599, 2066, 2027, 3710, 1999, 20980, 2874, 2001, 2043, 1045, 2253, 2000, 15898, 8566, 1012, 2023, 2173, 2003, 2092, 4276, 5983, 2012, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [11]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))


2022-04-11 13:49:59.017904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-11 13:49:59.017937: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-11 13:49:59.017955: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cet-herndon-jupyter): /proc/driver/nvidia/version does not exist
2022-04-11 13:49:59.018243: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)


2022-04-11 13:50:01.691991: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint 

In [14]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

In [15]:
model.save_pretrained("./sentiment")
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("./sentiment")


Some layers from the model checkpoint at ./sentiment were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./sentiment and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
labelString = ['Negative','Positive']

# test each review
# middle_index=len(reviews)//2

# first_half=reviews[:middle_index]
# sec_half=reviews[middle_index:]
# label_first_half = labels[:middle_index]
# label_sec_half = labels[middle_index:]
index = 0
for item1 in reviews:
    # compare to item1 in second half
    predict_input = tokenizer.encode(item1,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
    tf_output = loaded_model.predict(predict_input)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis=1)
    label = tf.argmax(tf_prediction, axis=1)
    label = label.numpy()
    print('---- COMPARE 1 ',labelString[labels[index]],'----', labelString[label[0]], '---- ', stars[index])
    print(item1)
    
    index = index + 1
    if index == 50:
        break
 

---- COMPARE 1  Positive ---- Positive ----  3
If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. 

The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.
---- COMPARE 1  Positive ---- Positive ----  5
I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycle. From the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.

For anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gym

---- COMPARE 1  Positive ---- Positive ----  5
I go to blow bar to get my brows done by natalie (brow specialist) which i highly recommend she is great does a great job on my eyebrows! But then i got a blow by victoria!! Wow i was impress i have thin, straight, dead hair and she left me with the biggest volume ive ever had!!! Tried another girl but didnt like it as much so victoria will be my girl for ever; very beautiful clean place!!!
---- COMPARE 1  Positive ---- Positive ----  5
My absolute favorite cafe in the city. Their black and white latte is probably the best I've ever had (not too sweet and just the right amount of foam), soups are always really good (even for non-soup people) and there's just a lot of space to do work. The noise level is perfect, the music is at a perfect level, and I always enjoy when patrons bring their dogs. I'll keep giving them my business for as long as I can.
---- COMPARE 1  Positive ---- Positive ----  5
HOLY SMOKES!

actual pumpkin pie mixed in wit

---- COMPARE 1  Positive ---- Positive ----  4
Food was good- atmosphere/decor is like a fishing lodge- menu is someplace between outback & bonefish. Went on a Groupon & all 3 of our meals & calamari appetizer that we ordered were cooked perfectly- we will def be back.
---- COMPARE 1  Negative ---- Positive ----  2
Straight to the point, it's cheap, it tastes and feels cheap

The good: price, location, dessert (edible food items are: baked chicken, fried chicken, mashed potatoes, Salad bar lol, meat and cheese bar, ravioli, pizza) also our waitress was bomb af

The meh: behind counter meat slicing station, shrimp (not even deveined), Mexican station

The bad: breakfast -worst eggs ever, actually worst, eggs. Le Benedict can't even cut through ham or English muffin (tried on all 3 occasions), steak, shrimp again, hell breaded items can't even, don't even. Decor it's the 1970 nightmare you can imagine

Tried 3 times and failed
---- COMPARE 1  Positive ---- Positive ----  4
The only reaso